# Using OpenVINO Docker containers for development and deployment


## Introduction

The purpose of this tutorial is to examine a sample application that was created using the [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit).  

This tutorial will go step-by-step to demonstrate [speech recognition demo](https://docs.openvinotoolkit.org/latest/omz_demos_speech_recognition_deepspeech_demo_python.html) using OpenVINO™ Docker container. Recognition is performed using a pre-trained network - Mozilla* DeepSpeech. We will also cover model download and conversion via Model Optimizer. The model is capable of recognizing the speech in English.  Inference will be executed using the same CPU(s) running this Jupyter* Notebook.


## Prerequisites

This tutorial requires the following:
- Unix* host
- Installed and run [Docker](https://docs.docker.com/engine/install/)* engine/service on the host


## Preparation

OpenVINO has a couple of dependencies which need to be present on your computer. Additionally, to install some of them, you need to have root/admin rights. This might not be desirable. Using Docker represents much cleaner way. Especially when there is an image prepared for you on [Docker Hub OpenVINO organization](https://hub.docker.com/r/openvino).

We have some types of Docker images depend on distribution:

*   **runtime** - contains Inference Engine and OpenCV runtime libs for each supported device (CPU, GNA, GPU, VPU)
*   **dev** - contains runtime part plus samples, demos and Python development tools: Model Optimizer, Post training Optimization tool (POT), Accuracy checker, Open Model Zoo tools (downloader, converter)
*   **data_runtime** - contains runtime part plus DL Streamer runtimes
*   **data_dev** - contains data_runtime and dev parts plus Media SDK, Speech Libraries and End-to-End Speech Demos

You can search OpenVINO Docker images on Docker Hub or via cli:





```
docker search --format "{{.Name}}: {{.Description}}" --no-trunc --limit 100  openvino | grep ^openvino/
```




Output will be like this:
```
openvino/workbench: OpenVINO™ DL Workbench is a web GUI to simplify DL models inference and tuning on Intel® devices.
openvino/ubuntu18_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu18_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu20_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/model_server: Intel® Distribution of OpenVINO™ Model Server Docker images
openvino/cvat_ui: Computer Vision Annotation Tool (CVAT) frontend UI image
openvino/ubuntu18_data_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/cvat_server: Computer Vision Annotation Tool (CVAT) annotation backend image
openvino/ubuntu18_dev_no_samples: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu18_data_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu18_model_server: [DEPRECATED] Please use openvino/model_server instead
openvino/ubuntu20_data_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/ubuntu20_data_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/winserver2019_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Windows Server Core base LTSC 2019
openvino/ubuntu20_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/ubuntu18_base_cpu: [DEPRECATED] Not maintained anymore
openvino/centos7_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for CentOS*
```



## Part 1. Using OpenVINO development Docker container for model conversion and execution.

### 1.	Pull data dev OpenVINO docker image and run it to download a model. 

**Note:** it will write a model on the host, please use /tmp folder.

`docker run` command will pull the 2021.3 release of openvino/ubuntu18_data_dev image from Docker Hub and run a container with `downloader.py` script to download `mozilla-deepspeech-0.6.1` model to `/tmp/model` folder on the host.

* `-u root` or `-u 0` option needs to save downloaded model in `/tmp/model` folder under root user.
* `--rm` option needs to remove container after execution.
* `-v /tmp/model:/tmp/model` option needs to mount host folder to the container.


In [ ]:
!mkdir /tmp/model
!docker run -u root --rm -v /tmp/model:/tmp/model openvino/ubuntu18_data_dev:2021.3 deployment_tools/open_model_zoo/tools/downloader/downloader.py --name mozilla-deepspeech-0.6.1 -o /tmp/model

Unable to find image 'openvino/ubuntu18_data_dev:2021.3' locally
2021.3: Pulling from openvino/ubuntu18_data_dev























Digest: sha256:cf45712ca65386eb64103ca0aa1d9058589cee02ced743097b25d296a6a776ca
Status: Downloaded newer image for openvino/ubuntu18_data_dev:2021.3
################|| Downloading mozilla-deepspeech-0.6.1 ||################

========== Downloading /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models.tar.gz
... 0%, 1024 KB, 811 KB/s, 1 seconds passed
... 0%, 2048 KB, 1082 KB/s, 1 seconds passed
... 0%, 3072 KB, 1213 KB/s, 2 seconds passed
... 0%, 4096 KB, 1301 KB/s, 3 seconds passed
... 0%, 5120 KB, 1286 KB/s, 3 seconds passed
... 0%, 6144 KB, 1347 KB/s, 4 seconds passed
... 0%, 7168 KB, 1422 KB/s, 5 seconds passed
... 0%, 8192 KB, 1508 KB/s, 5 seconds passed
... 0%, 9216 KB, 1583 KB/s, 5 seconds passed
... 0%, 10240 KB, 1654 KB/s, 6 seconds passed
... 0%, 11264 KB, 1719 KB/s, 6 seconds passed
... 1%, 12288 KB, 1784 KB/s, 6 seconds passed


### 2. Convert our model to the Intermediate Representation (IR)

OpenVINO™ toolkit introduces its own format of graph representation and its own operation set. A graph is represented with two files: an XML file and a binary file. This representation is commonly referred to as the Intermediate Representation or IR.

`docker run` comman will run a container with `converter.py` to convert the model stored in `/tmp/model` folder and save it to `/tmp/model/converted` folder. 

In [ ]:
!docker run -u 0 --rm -v /tmp/model:/tmp/model openvino/ubuntu18_data_dev:2021.3  deployment_tools/open_model_zoo/tools/downloader/converter.py  --name mozilla-deepspeech-0.6.1 -d /tmp/model -o /tmp/model/converted

========== Converting mozilla-deepspeech-0.6.1 to IR (FP16)
Conversion command: /usr/bin/python3 -- /opt/intel/openvino/deployment_tools/model_optimizer/mo.py --framework=tf --data_type=FP16 --output_dir=/tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16 --model_name=mozilla-deepspeech-0.6.1 --input_model=/tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/output_graph.pb '--freeze_placeholder_with_value=input_lengths->[16]' --input=input_node,previous_state_h,previous_state_c '--input_shape=[1,16,19,26],[1,2048],[1,2048]' --output=logits,cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/GatherNd,cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/GatherNd_1 --disable_nhwc_to_nchw

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/output_graph.pb
	- Path for generated IR: 	/tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16
	- IR output name:

### 3. Execute the Speech Recognition Demo

We need execute 3 steps to run speech demo in isolated environment:

1. We will build ctcdecode_numpy Python module, which implements CTC decoding in C++ for faster decoding.
2. Source OpenVINO setupvars.
3. Run Speech Recognition Demo with converted model, how_are_you_doing.wav audio sample stored in the OpenVINO package and pre/post-processing profile `-p mds06x_en` for Mozilla DeepSpeech v0.6.x model.

`docker run` command will run a container with `/bin/bash` session to execute 3 mentioned steps above.

In [ ]:
!docker run -u 0 --rm -v /tmp/model:/tmp/model openvino/ubuntu18_data_dev:2021.3 /bin/bash -c "cd deployment_tools/open_model_zoo/demos/speech_recognition_demo/python/ctcdecode-numpy && python3 -m pip install . && cd ../ && source /opt/intel/openvino/bin/setupvars.sh && python3 speech_recognition_demo.py -m /tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16/mozilla-deepspeech-0.6.1.xml -L /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/lm.binary -p mds06x_en -i /opt/intel/openvino/deployment_tools/demo/how_are_you_doing.wav"

Processing /opt/intel/openvino_2021.3.394/deployment_tools/open_model_zoo/demos/speech_recognition_demo/python/ctcdecode-numpy
  Created wheel for ctcdecode-numpy: filename=ctcdecode_numpy-0.2-cp36-cp36m-linux_x86_64.whl size=1367090 sha256=ab8fcb308eaf03f455d15414cd5e41034f4f2a837432a9e2375c7dac1a1c1268
  Stored in directory: /root/.cache/pip/wheels/ed/4e/96/8b168e73f4c6ad89de2e4cb3fcb62a3dc2a7b94c73476cef06
Successfully built ctcdecode-numpy
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
[setupvars.sh] OpenVINO environment initialized
100%|##########| 11/11 [00:00<00:00, 21.71it/s]
Loading, including network weights, IE initialization, LM, building LM vocabulary trie, loading audio: 1.9737495279987343 s
Audio file length: 3.405 s
MFCC time: 0.0034373460221104324 s
RNN time: 0.5094106969772838 s
Beam search time: 0.03883401898201555 s
Overall time: 2.525500933988951 s

Transcription and confidence score:
4.626835823059082	how are you doi

## Part 2. Set up OpenVINO runtime container to execute our demo

Let's imagine that we need to deploy this speech recognizer to production. OpenVINO provides a lightweight runtime container for this purpose. 



First, we need copy demo and audio sample from data dev image to the host `/tmp/demo` folder:

OpenVINO runtime image is 7 times lighter than data_dev image:

```
Image                              Hash           Size

openvino/ubuntu18_data_dev:2021.3  44513bd0f523  8.58GB

openvino/ubuntu18_runtime:2021.3   ab47b8e21031  1.11GB
```

In [ ]:
!docker run -u 0 --rm -v /tmp/demo:/tmp/demo openvino/ubuntu18_data_dev:2021.3 /bin/bash -c "cp -rf deployment_tools/open_model_zoo/demos/speech_recognition_demo/python /tmp/demo && cp -rf  /opt/intel/openvino/deployment_tools/demo/how_are_you_doing.wav /tmp/demo"

Second, let's try to run it on runtime OpenVINO image. We need repeat 3 steps mentioned above for data_dev image and install compiler, Python development tools to build pip wheel for ctcdecode-numpy and install `libsndfile1` library for reading/writing audio files:

In [ ]:
!docker run -u 0 --rm -v /tmp/demo:/tmp/demo -v /tmp/model:/tmp/model  openvino/ubuntu18_runtime /bin/bash -c "apt update && apt install -y gcc g++ python3-dev libsndfile1 && pip3 install -r /tmp/demo/python/requirements.txt && cd /tmp/demo/python/ctcdecode-numpy && pip3 install . && cd ../ && source /opt/intel/openvino/bin/setupvars.sh && python3 speech_recognition_demo.py -m /tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16/mozilla-deepspeech-0.6.1.xml -L /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/lm.binary -p mds06x_en -i /tmp/demo/how_are_you_doing.wav"



Get:1 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [24.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2119 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1410 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [399 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic/main amd64 Packages [1344 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:13 http://archive.ubun

Third, let's save our lightweight runtime image. In that case, we should execute some additional steps:


1.   Run a container in interactive mode and set a name for it to use it later:


```
docker run -u 0 -it --name speech --rm -v /tmp/demo:/tmp/demo -v /tmp/model:/tmp/model openvino/ubuntu18_runtime
```




2.   Execute our 3 prerequisite steps to setup environment for the speech sample.


```
apt update && apt install -y gcc g++ python3-dev libsndfile1 && pip3 install -r /tmp/demo/python/requirements.txt && cd /tmp/demo/python/ctcdecode-numpy && pip3 install . 
```



3.   Copy demo and model in the image



```
cp -rf /tmp/demo/python/* /opt/intel/openvino && cp -rf /tmp/demo/how_are_you_doing.wav /opt/intel/openvino && cp -rf /tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16/* /opt/intel/openvino && cp -rf /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/* /opt/intel/openvino
```



4.   Remove build artifacts



```
apt autoremove -y gcc g++ && rm -rf /var/lib/apt/lists/* && cd -
```



Now, our container is ready to be saved as a new image. Up another console and run `docker commit`:

```sh
docker commit speech ubuntu18_runtime:speech
```




Created image (Mozilla* DeepSpeech is a very big, plus 2 Gb to the runtime image):
```
Name                                 Hash                Size
ubuntu18_runtime:speech              6451dd798120       3.08GB
```


We can run the demo as non-root user with our speech image: 


```
docker run --rm ubuntu18_runtime:speech /bin/bash -c "source bin/setupvars.sh && python3 speech_recognition_demo.py -m mozilla-deepspeech-0.6.1.xml -L lm.binary -p mds06x_en -i how_are_you_doing.wav"
```



Of course, you can run a container and inference on different Intel devices like iGPU (Intel® Processor Graphics), NCS2(Intel® Neural Compute Stick 2) or HDDL (Intel® Vision Accelerator Design with Intel® Movidius™ VPUs). 

To enable GPU access, add `--device /dev/dri` to `docker run` command and `-d GPU` to demo command line. GPU will be shared as device with container:


```
docker run --rm --device /dev/dri ubuntu18_runtime:speech /bin/bash -c "source bin/setupvars.sh && python3 speech_recognition_demo.py -m mozilla-deepspeech-0.6.1.xml -L lm.binary -p mds06x_en -i how_are_you_doing.wav -d GPU"
```


To run inference on the NCS2 USB stick, add `--device-cgroup-rule='c 189:* rmw' -v /dev/bus/usb:/dev/bus/usb` to `docker run` command, and `-d MYRIAD` to demo command line. Mount USB stick will be shared as volume and set up the rule to the cgroup allowed 189 USB char devices the following access rights (r - read, m – mknod – create a device and w - write):

```
docker run --rm --device-cgroup-rule='c 189:* rmw' -v /dev/bus/usb:/dev/bus/usb ubuntu18_runtime:speech /bin/bash -c "source bin/setupvars.sh && python3 speech_recognition_demo.py -m mozilla-deepspeech-0.6.1.xml -L lm.binary -p mds06x_en -i how_are_you_doing.wav -d MYRIAD"
```

You can find more information about how to run a container with access to a specific device on our [Docker CI GitHub repo](https://github.com/openvinotoolkit/docker_ci/blob/master/get-started.md#run-a-container)

## Summary

In this article, we briefly introduced the speech recognition demo using OpenVINO Docker images. Of course, there is much more to try. We hope that this article has motivated you to try it yourself and maybe continue to explore all the possibilities of OpenVINO Docker images.



## References



*   [Available OpenVINO Docker images](https://github.com/openvinotoolkit/docker_ci#prebuilt-images)
*   [Docker CI framework for Intel® Distribution of OpenVINO™ toolkit](https://github.com/openvinotoolkit/docker_ci). The Framework can generate a Dockerfile, build, test, and deploy an image with the Intel® Distribution of OpenVINO™ toolkit. You can reuse available Dockerfiles, add your layer and customize the image of OpenVINO™ for your needs.

